In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [16]:
from transformers import LlamaConfig, LlamaForCausalLM, AutoTokenizer

In [4]:
half_config = LlamaConfig.from_pretrained('fpf-7b-32k/checkpoint-7400', 
                                             num_hidden_layers = 4)
half_config

LlamaConfig {
  "_name_or_path": "mesolitica/llama-7b-hf-16384-fpf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 32768,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 4,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [5]:
small = LlamaForCausalLM(half_config)

In [6]:
small

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm

In [8]:
sum(p.numel() for p in small.parameters())

1071681536

In [7]:
model = LlamaForCausalLM.from_pretrained('fpf-7b-32k/checkpoint-7400')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
import torch.nn as nn

def copy_layers(src_layers, dest_layers, layers_to_copy):
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())

layers_to_copy = [0,1,2,3]

copy_layers(model.model.layers, small.model.layers, layers_to_copy)

In [10]:
small.model.embed_tokens.load_state_dict(model.model.embed_tokens.state_dict())

<All keys matched successfully>

In [11]:
small.model.norm.load_state_dict(model.model.norm.state_dict())

<All keys matched successfully>

In [12]:
small.lm_head.load_state_dict(model.lm_head.state_dict())

<All keys matched successfully>

In [14]:
small.save_pretrained("1b")

In [18]:
tokenizer = AutoTokenizer.from_pretrained('fpf-7b-32k/checkpoint-7400')
tokenizer.save_pretrained("1b")

('1b/tokenizer_config.json', '1b/special_tokens_map.json', '1b/tokenizer.json')

In [19]:
!ls -lh 1b

total 4.0G
-rw-r--r-- 1 ubuntu ubuntu  633 Sep 17 06:14 config.json
-rw-r--r-- 1 ubuntu ubuntu  132 Sep 17 06:14 generation_config.json
-rw-r--r-- 1 ubuntu ubuntu 4.0G Sep 17 06:14 pytorch_model.bin
-rw-r--r-- 1 ubuntu ubuntu  414 Sep 17 06:14 special_tokens_map.json
-rw-r--r-- 1 ubuntu ubuntu 1.8M Sep 17 06:14 tokenizer.json
-rw-r--r-- 1 ubuntu ubuntu  725 Sep 17 06:14 tokenizer_config.json
